<a href="https://colab.research.google.com/github/StuckInAnEndlessFebruary/Image_processing_homework/blob/main/Image_processing_homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>